In [6]:
import pandas as pd 
import os 
import numpy as np 
from scipy.spatial.distance import cdist
import folium
import re
from scipy.spatial import distance_matrix
from sklearn.metrics import silhouette_score
from sklearn.cluster import KMeans


In [7]:
#selecção de dados de input
centro = '8819009'
codigo_prestador='2800'
codigo_prestador_2=''
giros_excluir=['2800H','2800I']
cp_validation =['2800','2805']
data_inicio ='20240101'


Importação de ficheiros da pasta FTP

In [3]:
# folder_path = os.path.join(os.path.expanduser("~"), "Downloads/Pit_stop/Dados_centros")

# files = [file for file in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, file))]


# combined_df = pd.DataFrame()


# for file in files:
#     file_path = os.path.join(folder_path, file)
    
    
#     df = pd.read_csv(file_path, encoding="latin-1", delimiter=';')
    
    
#     combined_df = pd.concat([combined_df, df], ignore_index=True)



importação de ficheiros da base de dados

In [8]:
import psycopg2
import psycopg2.extras
import os
#import pygwalker as pyg

def conecta_db():
      con = psycopg2.connect(host='localhost', 
                         database='ctt2024',
                         user='postgres', 
                         password='postgres')
      return con


def criar_db(sql):
    con = conecta_db()
    cur = con.cursor()
    cur.execute(sql)
    con.commit()
    con.close()
    
    
def inserir_db(sql):
    con = conecta_db()
    cur = con.cursor()
    try:
        cur.execute(sql)
        con.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        con.rollback()
        cur.close()
        return 1
    cur.close()

In [9]:
query_filtro_data = f"SELECT *, CAST(SUBSTRING(data_criacao, 7, 4) || SUBSTRING(data_criacao, 4, 2) || SUBSTRING(data_criacao, 1, 2) AS INTEGER) AS sortable_date_int FROM expresso_2024 WHERE centro = '{centro}' AND CAST(SUBSTRING(data_criacao, 7, 4) || SUBSTRING(data_criacao, 4, 2) || SUBSTRING(data_criacao, 1, 2) AS INTEGER) > {int(data_inicio)};"
con = conecta_db()
cur = con.cursor(cursor_factory=psycopg2.extras.DictCursor)
query = query_filtro_data
cur.execute(query)
results_expresso_ = cur.fetchall()


In [10]:
con = conecta_db()
cur = con.cursor(cursor_factory=psycopg2.extras.DictCursor)
query = 'SELECT "CP7", "POINT_X", "POINT_Y" FROM cp7coords'
cur.execute(query)
df_sig = cur.fetchall()

Limpeza dos dados


In [11]:
# -- limpeza do dataset 
# -- df = combined_df

df = pd.DataFrame(results_expresso_, columns=['data_criacao', 'centro', 'Giro', 'LOPTICA', 'JANELA_HORARIA', 'NOME', 'MORADA', 'CP', 'LOCALIDADE', 'COD_T_EVEN', 'DATA_EVENTO', 'LATITUDE', 'LONGITUDE','sortable_date_int'])
df.columns=[x.lower().strip() for x in df.columns]
df.drop_duplicates()
df['centro'] = df['centro'].astype(str).str.replace('.0',"")
df['centro'] = df['centro'].str.strip()
df['giro'] = df['giro'].str.replace(' ', '').str.strip().str[:5]
df['cp'] = df['cp'].str.replace(' ', '').str.strip()
df['longitude'] = df['longitude'].str.replace('\n', '').str.strip()
df=df[df.latitude != '0']
df=df[df.latitude != '0.0']
df = df[df['latitude'] != '']
df=df[df.cod_t_even == 'I']
df = df[df['cp'].str.len() >= 8]


In [12]:
# remove os fins de semana 
df['data_criacao'] = pd.to_datetime(df['data_criacao'], format='%d/%m/%Y')

df = df[df['data_criacao'].dt.dayofweek < 5]

In [14]:
df=df[df['centro']==centro]


In [ ]:
# -- considera apenas cp7 que ocorreram várias vezes --#

giro_unique_cp7 = df.groupby('cp')['data_criacao'].nunique().reset_index(name='N_Unique_cp7')

giro_unique_cp7 = giro_unique_cp7[giro_unique_cp7['N_Unique_cp7'] >= 1]

giro_unique_cp7=np.array(giro_unique_cp7['cp'].unique())

df = df[df['cp'].isin(giro_unique_cp7)]

In [ ]:
#-- excluir cp7 efetuados por várias rotas --#

unique_routes_by_cp = df.groupby('cp')['giro'].nunique().reset_index(name='Unique_Routes_Count')
unique_routes_by_cp = unique_routes_by_cp.sort_values(by='Unique_Routes_Count', ascending=False)

unique_routes_by_cp_filtered = unique_routes_by_cp[unique_routes_by_cp['Unique_Routes_Count'] ==1]

lista_excl_cp7=np.array(unique_routes_by_cp_filtered['cp'].unique())

df = df[df['cp'].isin(lista_excl_cp7)]

cp7_unique_counts = df['cp'].value_counts()

In [ ]:
# selecção de giros 
#codigo_prestador='2405'
#codigo_prestador_2='2410'
#giros_excluir=['2405D','1000B']

giro_unique_counts = df.groupby('giro')['data_criacao'].nunique().reset_index(name='N_Unique_Data_Criacao')

giro_unique_counts = giro_unique_counts[
    ((giro_unique_counts['giro'].str[:4] == codigo_prestador) |
     (giro_unique_counts['giro'].str[:4] == codigo_prestador_2)) &
    (~giro_unique_counts['giro'].apply(lambda x: any(x.startswith(prefix) for prefix in giros_excluir)))
]

giro_unique_counts_average = giro_unique_counts['N_Unique_Data_Criacao'].mean()
giro_unique_counts_average

giro_unique_counts=giro_unique_counts[giro_unique_counts['N_Unique_Data_Criacao'] > 0]
giro_unique_counts

print(giro_unique_counts)

lista_val_giro=np.array(giro_unique_counts['giro'].unique())

print(lista_val_giro)

df = df[df['giro'].isin(lista_val_giro)]

    giro  N_Unique_Data_Criacao
0  2800A                     27
1  2800B                     26
2  2800C                     27
3  2800D                     26
4  2800E                     27
5  2800F                      1
6  2800G                     17
['2800A' '2800B' '2800C' '2800D' '2800E' '2800F' '2800G']


In [ ]:
# -- validação do cp4 -- 
# unique_cp4_values = df['cp'].str[:4].unique()

# cp_counts_by_unique_cp4 = {}
# for cp4_value in unique_cp4_values:
#     cp_counts_by_unique_cp4[cp4_value] = df[df['cp'].str[:4] == cp4_value]['cp'].count()

# cp_counts = pd.DataFrame(list(cp_counts_by_unique_cp4.items()), columns=['CP4_Value', 'Count'])

# cp_counts = cp_counts[cp_counts['Count'] > 100]  


# cp_validation = np.array(cp_counts['CP4_Value'])

#cp_validation =['2405','2410','2400','2415','2420']

def expand_values(cp_values):
    expanded_values = []
    
    for value in cp_values:
        value = int(value)
        expanded_values.extend([str(value + i) for i in range(5)])
    
    return expanded_values

expanded_validation = expand_values(cp_validation)

#print(cp_counts)
print(cp_validation)
print(expanded_validation)

print(len(cp7_unique_counts))

['2800', '2805']
['2800', '2801', '2802', '2803', '2804', '2805', '2806', '2807', '2808', '2809']
3868


Dados Sig Postal

In [ ]:


# file_path = os.path.join(os.path.expanduser("~"), "Downloads/Pit_stop/Sig_postal")


# files = [file for file in os.listdir(file_path) if os.path.isfile(os.path.join(file_path, file))]

# for file in files:

#     file_path = os.path.join(file_path, file)
#     df_sig = pd.read_csv(file_path, encoding="latin-1", delimiter=';')
    


In [ ]:
df_sig=pd.DataFrame(df_sig,columns=['cp7','point_x','point_y'])

df_sig = df_sig.apply(lambda x: x.astype(str).str.replace(' ', '').str.strip() if pd.notnull(x.name) else x)

df_sig['cp7'] = df_sig['cp7'].astype(str)
df_sig['point_x'] = df_sig['point_x'].str.replace('\n', '').str.strip()
df_sig['point_y'] = df_sig['point_y'].str.replace('\n', '').str.strip()
df_sig['point_x'] = df_sig['point_x'].str.replace('.', ',').str.strip()
df_sig['point_y'] = df_sig['point_y'].str.replace('.', ',').str.strip()
df_sig=df_sig[df_sig.point_x != '0']
df_sig=df_sig[df_sig.point_y != '0']
df_sig=df_sig[df_sig.point_x != '0.0']
df_sig=df_sig[df_sig.point_y != '0.0']
df_sig = df_sig[df_sig['point_x'] != '']
df_sig = df_sig[df_sig['point_y'] != '']
df_sig = df_sig[df_sig['point_x'] != 'nan']
df_sig = df_sig[df_sig['point_y'] != 'nan']
df_sig = df_sig.dropna()


In [ ]:
df_sig = df_sig[df_sig['cp7'].str[:4].isin(expanded_validation)]

In [ ]:

df['data_criacao'] = pd.to_datetime(df['data_criacao'], dayfirst=True)

result = df.groupby(['cp', 'giro']).size().reset_index(name='count')

idx = result.groupby('cp')['count'].idxmax()

df_result = result.loc[idx].reset_index(drop=True)



In [ ]:
merged_df = pd.merge(df_sig, df_result[['cp', 'giro']], how='left', left_on='cp7', right_on='cp')

merged_df.drop(['cp'], axis=1, inplace=True)



In [ ]:
# selected_columns = ['cp7', 'cp4', 'point_x', 'point_y', 'giro']
# selected_df = merged_df[selected_columns]

# downloads_path = os.path.join(os.path.expanduser("~"), "Downloads/Pit_stop/resultados")


# file_path = os.path.join(downloads_path, f'{codigo_prestador}.xlsx')
# selected_df.to_excel(file_path, index=False)

# print(f"File saved at: {file_path}")

Preencher os cp7 sem rota 

In [ ]:
#calcular a distância entre pontos e afetar os cp7 sem rota aos que tem rota mais próximos


merged_df['point_x'] = merged_df['point_x'].str.replace(',', '.').astype(float)
merged_df['point_y'] = merged_df['point_y'].str.replace(',', '.').astype(float)

def fill_giro(row):
    if pd.isna(row['giro']):
        
        non_null_rows = merged_df[merged_df['giro'].notna()]

        
        distances = cdist(np.array([[row['point_x'], row['point_y']]]), non_null_rows[['point_x', 'point_y']], metric='cityblock') 

        
        closest_index = np.argmin(distances)

        
        return non_null_rows.iloc[closest_index]['giro']
    else:
        return row['giro']

merged_df['giro'] = merged_df.apply(fill_giro, axis=1)

print(merged_df)


          cp7   point_x    point_y   giro
0    2800-001 -9.160299  38.680930  2800D
1    2800-002 -9.160122  38.682186  2800D
2    2800-009 -9.153465  38.681730  2800A
3    2800-010 -9.155036  38.680813  2800E
4    2800-011 -9.153449  38.681690  2800A
..        ...       ...        ...    ...
609  2809-009 -9.156847  38.676945  2800E
610  2809-010 -9.155653  38.677578  2800E
611  2809-011 -9.165897  38.672897  2800E
612  2809-013 -9.174791  38.677334  2800C
613  2809-015 -9.169362  38.670925  2800C

[614 rows x 4 columns]


In [ ]:
from sklearn.metrics import silhouette_samples

labels = merged_df.giro
data_points = merged_df[['point_x', 'point_y']]

individual_silhouette_scores = silhouette_samples(data_points, labels)

merged_df['individual_silhouette'] = individual_silhouette_scores

print(merged_df[['cp7', 'giro', 'individual_silhouette']])


          cp7   giro  individual_silhouette
0    2800-001  2800D              -0.152651
1    2800-002  2800D              -0.175707
2    2800-009  2800A               0.463950
3    2800-010  2800E              -0.335987
4    2800-011  2800A               0.461473
..        ...    ...                    ...
609  2809-009  2800E               0.049666
610  2809-010  2800E               0.007889
611  2809-011  2800E              -0.348027
612  2809-013  2800C               0.518753
613  2809-015  2800C               0.514224

[614 rows x 3 columns]


In [ ]:
# specific_point = merged_df[merged_df['cp7'] == '2820-306']

# print(specific_point[['cp7', 'cp4', 'giro', 'individual_silhouette']])


In [ ]:
# tratamento aos pontos com atribuição duvidosa

import pandas as pd
from scipy.spatial import distance_matrix
from sklearn.metrics import silhouette_score
from sklearn.cluster import KMeans


giro_centers = merged_df.groupby('giro')[['point_x', 'point_y']].mean()

def assign_giro(row, giro_centers):
    point_coords = row[['point_x', 'point_y']]
    distances = distance_matrix([point_coords], giro_centers)[0]
    closest_giro = giro_centers.index[distances.argmin()]
    return closest_giro

merged_df['correct_giro'] = merged_df.apply(lambda row: assign_giro(row, giro_centers), axis=1)

merged_df['distance_to_correct_centroid'] = merged_df.apply(
    lambda row: distance_matrix([row[['point_x', 'point_y']]], [giro_centers.loc[row['giro']]])[0][0], axis=1
)

silhouette_avg = silhouette_score(merged_df[['point_x', 'point_y']], merged_df['correct_giro'])

print(f"Índice de Silhueta para os giros corretos: {silhouette_avg}")

limiar_silhueta = -0.2
if silhouette_avg < limiar_silhueta:
    percentual_alteracoes_permitido = 100  
else:
    percentual_alteracoes_permitido = 0 
    
merged_df = merged_df.sort_values(by='distance_to_correct_centroid', ascending=False)

merged_df['change'] = 0
for giro in merged_df['giro'].unique():
    giro_indices = merged_df[merged_df['giro'] == giro].index
    num_changes_allowed = int((percentual_alteracoes_permitido / 100) * len(giro_indices))
    merged_df.loc[giro_indices[:num_changes_allowed], 'change'] = 1

merged_df.loc[merged_df['change'] == 1, 'giro'] = merged_df.loc[merged_df['change'] == 1, 'correct_giro']

merged_df = merged_df.drop(['correct_giro', 'distance_to_correct_centroid', 'change'], axis=1)


Índice de Silhueta para os giros corretos: 0.27919067022000527


In [ ]:


giro_centers = merged_df.groupby('giro')[['point_x', 'point_y']].mean()

def assign_giro(row, giro_centers):
    point_coords = row[['point_x', 'point_y']]
    distances = distance_matrix([point_coords], giro_centers)[0]
    closest_giro = giro_centers.index[distances.argmin()]
    return closest_giro

merged_df['correct_giro'] = merged_df.apply(lambda row: assign_giro(row, giro_centers), axis=1)

silhouette_avg = silhouette_score(merged_df[['point_x', 'point_y']], merged_df['correct_giro'])
print(f"Average Silhouette Score: {silhouette_avg}")

limiar_silhueta = -0.2

points_to_reassign = merged_df[merged_df['individual_silhouette'] < limiar_silhueta]

merged_df.loc[points_to_reassign.index, 'giro'] = points_to_reassign['correct_giro']

merged_df = merged_df.drop(['correct_giro'], axis=1)



Average Silhouette Score: 0.33868085845719914


In [ ]:

df_final=merged_df

m = folium.Map(location=[df_final['point_y'].mean(), df_final['point_x'].mean()], zoom_start=12, tiles='OpenStreetMap')

color_mapping = {}
unique_rotas = df_final['giro'].unique()

fixed_colors = ["#3498DB", "#E74C3C", "#F39C12", "#8E44AD",
                "#34495E", "#D35400", "#1ABC9C", "#E67E22", "#BDC3C7", "#34495E",
                "#16A085", "#F1C40F", "#2C3E50", "#58D68D", "#AF7AC5",
                "#3498DB", "#27AE60", "#8E44AD"]



for index, rota in enumerate(unique_rotas):
    color_mapping[rota] = fixed_colors[index % len(fixed_colors)]


rota_feature_groups = {}
for rota in unique_rotas:
    rota_feature_groups[rota] = folium.FeatureGroup(name=rota) 

for _, row in df_final.iterrows():
    rota_color = color_mapping.get(row['giro'], '#1787FE')  
    folium.RegularPolygonMarker(  
        location=[row['point_y'], row['point_x']],
        number_of_sides=10,  
        radius=3,
        popup=f"CP7: {row['cp7']}, Giro: {row['giro']}",
        color=rota_color,
        fill=True,
        fill_color=rota_color
    ).add_to(rota_feature_groups[row['giro']])  


for rota, feature_group in rota_feature_groups.items():
    m.add_child(feature_group)


folium.LayerControl().add_to(m)

m


In [ ]:
# selected_columns = ['cp7', 'cp4', 'point_x', 'point_y', 'giro']
# selected_df = merged_df[selected_columns]

# downloads_path = os.path.join(os.path.expanduser("~"), "Downloads/Pit_stop/resultados")

# file_path = os.path.join(downloads_path, f'{codigo_prestador}.xlsx')
# selected_df.to_excel(file_path, index=False)

# print(f"File saved at: {file_path}")